
**참고자료**
<br>
<br>
 * https://somjang.tistory.com/entry/Keras%EA%B8%B0%EC%82%AC-%EC%A0%9C%EB%AA%A9%EC%9D%84-%EA%B0%80%EC%A7%80%EA%B3%A0-%EA%B8%8D%EC%A0%95-%EB%B6%80%EC%A0%95-%EC%A4%91%EB%A6%BD-%EB%B6%84%EB%A5%98%ED%95%98%EB%8A%94-%EB%AA%A8%EB%8D%B8-%EB%A7%8C%EB%93%A4%EC%96%B4%EB%B3%B4%EA%B8%B0
<br>
<br>
 * https://sig413.tistory.com/5
<br>
<br>
 * https://www.youtube.com/watch?v=7GUoDHxN5NM

# 네이버 기사 감정 분석

## 데이터 불러오기
<br>

**데이터 특성 정의( 각각의 데이터가 어떤 특성을 가지는지? )**

    ▶ date : 등록된 기사별 날짜
    ▶ title : 기사 제목
    ▶ source : 언론사
    ▶ contents : 기사 내용 요약본
    ▶ link : 사이트 주소

※ : 네이버 뉴스 기준으로 Test... ( 구글 뉴스데이터와 병합해서 전처리 실시 예정 )

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import datetime, time
import re
import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 시각화 글자 깨짐 방지
import os

if os.name == 'posix':
    plt.rc("font", family="Malgun Gothic")
else:
    plt.rc("font", family="Malgun Gothic")

In [3]:
matplotlib.rc('font',family='NanumGothic')

# 한글 폰트 적용
matplotlib.rc('axes', unicode_minus = False) # True

# 2022 / 04 / 21 일자 스크래핑 데이터
# df_news = pd.read_csv('./news_data.csv', engine= 'python',encoding = 'utf-8')
df_news = pd.read_csv('./Data_Set/2019.05.01~2022.05.01_DGB대구은행.csv', engine= 'python',encoding = 'utf-8')

# (행, 열)
print(df_news.shape)
df_news.head(1)

(28363, 5)


,Unnamed: 0,date,title,press,contents
0,0,2019.05.01.,"DGB 대구은행파크 건설에 뒷돈?…경찰, 대구 건설본부 압색",노컷뉴스,프로축구 대구 FC 전용구장인 DGB 대구은행파크를 건설하는 과정에 금품이 오갔다는...


In [4]:
# 데이터 구조 파악시 용이
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28363 entries, 0 to 28362
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  28363 non-null  int64 
 1   date        28363 non-null  object
 2   title       28363 non-null  object
 3   press       28363 non-null  object
 4   contents    28363 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.1+ MB


# 데이터 전처리

## 이상치와 결측치 제거 / 데이터 특성(구조) 확인
    ▶ 필요없는 컬럼 제거
    ▶ 컬럼명과 순서 변경 
    ▶ 데이터 구조 파악 - info, head
    
    ▶ 우선 'DGB대구은행'과 필요없는 언론사는 버리고 인덱스번호도 초기화 시켜야 함.
    예를 들면, 축구관련 기사

    제목과 요약본 기준
    '감독' / '신태용' / 'DGB대구은행파크' / '대표팀' / '경기' / '스포츠' / '축구'
    참고 사이트 : https://jimmy-ai.tistory.com/92
※ : 수치별 데이터가 아니므로 이상치는 굳이 확인할 필요가 없다고 판단

In [5]:
# 필요한 데이터만 추출 / 컬럼명 변경

df_news = df_news.loc[:,['date', 'title', 'press', 'contents']]
df_news.rename(columns={'source' : 'press'}, inplace=True)

In [6]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28363 entries, 0 to 28362
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      28363 non-null  object
 1   title     28363 non-null  object
 2   press     28363 non-null  object
 3   contents  28363 non-null  object
dtypes: object(4)
memory usage: 886.5+ KB


In [7]:
df_news.head(1)

,date,title,press,contents
0,2019.05.01.,"DGB 대구은행파크 건설에 뒷돈?…경찰, 대구 건설본부 압색",노컷뉴스,프로축구 대구 FC 전용구장인 DGB 대구은행파크를 건설하는 과정에 금품이 오갔다는...


In [8]:
# 행별로 null값 존재 유무 파악 --> 결측치 contents쪽에 3개 존재.
df_news.isnull().sum(axis = 0)

date        0
title       0
press       0
contents    0
dtype: int64

In [9]:
df_news[df_news['contents'].str.contains('오류', na=False)][:5]

,date,title,press,contents


In [10]:
# 불필요한 내용 삭제 / contents의 길이가 20이하 또는 title의 길이가 10 이하인 데이터
df_news.head()

# 51개 스크래핑 오류

print('이상치 제거 전:',len(df_news[df_news['contents'].str.contains('오류', na=False)]))

for i in df_news[df_news['contents'].str.contains('오류', na=False)].index:
    df_news.drop(i, axis=0, inplace=True)
    
print('이상치 제거 후:',len(df_news[df_news['contents'].str.contains('오류', na=False)]))

이상치 제거 전: 0
이상치 제거 후: 0


In [11]:
df_news.head(1)

,date,title,press,contents
0,2019.05.01.,"DGB 대구은행파크 건설에 뒷돈?…경찰, 대구 건설본부 압색",노컷뉴스,프로축구 대구 FC 전용구장인 DGB 대구은행파크를 건설하는 과정에 금품이 오갔다는...


In [12]:
len(df_news)

28363

# 학습 --> 중복 제거 해봄.

- df_news_overlap --> 제목 기사 중복 제거 O
- df_news_overlap --> 제목 기사 중복 제거 X

In [13]:
df_news

,date,title,press,contents
0,2019.05.01.,"DGB 대구은행파크 건설에 뒷돈?…경찰, 대구 건설본부 압색",노컷뉴스,프로축구 대구 FC 전용구장인 DGB 대구은행파크를 건설하는 과정에 금품이 오갔다는...
1,2019.05.01.,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",한스경제,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4..."
2,2019.05.01.,"대구FC, 3일 상주전 홈경기 대세존 운영",뉴스웨이,대구FC가 오는 3일 오후 7시 30분 DGB대구은행파크에서 열리는 하나원큐 K리그...
3,2019.05.01.,충청권이 타 지역 지방은행 각축장이 된 까닭,충청투데이,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...
4,2019.05.01.,대전에 문 연 대구은행…지역 반응은 '글쎄',충청투데이,DGB대구은행이 대전지역 진출을 본격화한 가운데 다소 냉담한 반응이 흘러나오고 있다...
...,...,...,...,...
28358,7일 전,"지방금융 3社, 순이익 증가율 4대 금융도 제쳐",한국경제,대구은행의 모기업인 DGB금융지주도 같은 날 1분기 순이익이 1622억원으로 집계됐...
28359,7일 전,"BNK·DGB금융그룹, 올해 1분기 실적 발표…당기순익 큰 폭 증가",대한금융신문,주요 계열서인 DGB대구은행의 올해 1분기 순이익은 전년동기 대비 29.7% 증가한...
28360,7일 전,"DGB금융그룹, 1분기 순이익 1622억…전년比 31% 증가",팍스경제TV,주력 계열사인 DGB대구은행의 1분기 순이익은 전년 동기 대비 29.7% 증가한 1...
28361,7일 전,"지방금융 3사, 역대급 호실적 행진…당분간 이자익 증가 전망",이뉴스투데이,주력 계열사인 DGB대구은행의 1분기 순이익은 전년 동기 대비 29.7% 증가한 1...


In [14]:
df_news_overlap = df_news.copy()
df_news_overlap.drop_duplicates(['contents'], keep = False, inplace=True)
df_news_overlap.drop_duplicates(['title'], keep = False, inplace=True)

In [15]:
# 중복행 제거 --> 분석하는데 안 좋을 수도 있으니 신중한 고려가 필요함.
# 참고 : https://sy-log.tistory.com/60

print(df_news.shape)
print('중복행 제거 전 데이터 :', len(df_news))
# 중복행 살림.
print('중복행 갯수 : %d개' % df_news.duplicated().sum())
print('중복행 제거 후 데이터 :', len(df_news.drop_duplicates(['title'], keep = False)))

(28363, 4)
중복행 제거 전 데이터 : 28363
중복행 갯수 : 0개
중복행 제거 후 데이터 : 28363


In [16]:
# df.query('주소.str.contains("마포|은평")', engine='python'
# Series 내장함수 : str.contains() 

# 제목
remove_title = list(df_news.query('title.str.contains("부상|선수|승점|구장|리그|DGB대구은행파크|FC|감독|신태용|연승|대표팀|친선경기|스포츠|축구|세징야", na=False)', engine = 'python').index)
df_news.drop(remove_title, axis=0, inplace = True)

# 내용 요약본
remove_contents = list(df_news.query('contents.str.contains("부상|선수|승점|구장|리그|DGB대구은행파크|FC|감독|신태용|연승|대표팀|친선경기|스포츠|축구|세징야", na=False)', engine = 'python').index)
df_news.drop(remove_contents, axis=0, inplace = True)


### 결측치 처리 contents 컬럼에 3개 존재 --> 3개의 데이터 삭제

- 2021.05.01 ~ 2022.05.01 결측치 존재
- 2019.05.01 ~ 2022.05.01 결측치 없음

In [17]:
df_news.count()

date        19405
title       19405
press       19405
contents    19405
dtype: int64

In [18]:
print(df_news.isnull().sum())
df_news.dropna(axis=0, inplace = True)

date        0
title       0
press       0
contents    0
dtype: int64


In [19]:
df_news.shape

(19405, 4)

In [20]:
# 인덱스 번호 새로 부여.
df_news.reset_index(inplace=True)
df_news.drop(['index'], axis=1, inplace=True)

In [21]:
df_news.head()

,date,title,press,contents
0,2019.05.01.,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",한스경제,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4..."
1,2019.05.01.,충청권이 타 지역 지방은행 각축장이 된 까닭,충청투데이,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...
2,2019.05.01.,대전에 문 연 대구은행…지역 반응은 '글쎄',충청투데이,DGB대구은행이 대전지역 진출을 본격화한 가운데 다소 냉담한 반응이 흘러나오고 있다...
3,2019.05.01.,DGB대구은행 대전 1호 점포 개점...전국 확대,대구MBC,DGB대구은행이 대전에 점포를 열었습니다. DGB대구은행은 어제 대전시 서구 문예로...
4,2019.05.01.,"DGB대구은행, 대전지점 개점",경상매일신문,"DGB대구은행이 30일 대전지점(대전광역시 서구 문예로 65)을 개점하고, 국토 및..."


In [22]:
df_news.count()

date        19405
title       19405
press       19405
contents    19405
dtype: int64

## 날짜 데이터 정제
    ▶ 연월일로만 표현 ( 시간 제거 )
    ▶ 연 / 월 / 일 3개의 파생변수 생성
    ▶ 연 / 일 보다는 월별로 각 언론사의 'DGB대구은행'에 관련된 기사가 얼마나 많은지? 파악하기 위함. --> 긍정 / 부정 개수로 이어짐.

In [23]:
# 연도별 월별 일별로 
df_news['date']

0        2019.05.01.
1        2019.05.01.
2        2019.05.01.
3        2019.05.01.
4        2019.05.01.
            ...     
19400           7일 전
19401           7일 전
19402           7일 전
19403           7일 전
19404           7일 전
Name: date, Length: 19405, dtype: object

In [24]:
now = time.localtime()

# 테스트 용 / 참고 - https://wikidocs.net/104836
print(datetime.datetime.now())
print('{}년 {}월 {}일'.format(now.tm_year, now.tm_mon, now.tm_mday-1))
print(pd.to_datetime(df_news['date'][0]) - datetime.timedelta(days = 6))

2022-06-03 14:56:59.502470
2022년 6월 2일
2019-04-25 00:00:00


In [25]:
# 2022-04-21 17:45 일자 데이터 자료 ( 시각은 무시 )
scrapping_date = datetime.date(2022, 5, 6)

idx_list = list()

for i in range(len(df_news['date'])):
    if '일' in df_news['date'][i]:
        idx_list.append(i)
    elif '시간' in df_news['date'][i]:
        idx_list.append(i)
# df_news['date'][i]
# 몇 일전 / 몇 시간전 인덱스 번호

In [26]:
# 확인
df_news.loc[idx_list, ['date']][:3]

,date
19307,4일 전
19308,4일 전
19309,4일 전


In [27]:
# 시간 전 / 몇일 전 --> 숫자로 변환

for i in idx_list:
    
    trans_day = int(re.findall('\d+', df_news['date'][i])[0]) 

    if '시간' in df_news['date'][i]:
        df_news['date'][i] = scrapping_date - datetime.timedelta(days=1)
    else:
        df_news['date'][i] = scrapping_date - datetime.timedelta(days=trans_day+1)

In [28]:
df_news['date'] = pd.to_datetime(df_news['date'])

In [29]:
df_news['year'] = df_news['date'].dt.year
df_news['month'] = df_news['date'].dt.month
df_news['day'] = df_news['date'].dt.day

In [30]:
df_news.head(1)

,date,title,press,contents,year,month,day
0,2019-05-01,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",한스경제,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4...",2019,5,1


In [31]:
# 데이터 자료 시간에서 마이너스해주면 날짜가 나옴.
# 이것을 다시 대입해주면 날짜 데이터 정제 완료

# 컬럼 제거 axis = 1 --> 열 / axis = 0 --> 행
# df_news.drop(['date_day'], axis=1, inplace=True)

In [32]:
# 참고 : https://hogni.tistory.com/51
# 컬럼 순서 정렬
df_news = df_news[['date', 'year', 'month', 'day', 'title', 'press', 'contents']]

In [33]:
df_news.head(3)

,date,year,month,day,title,press,contents
0,2019-05-01,2019,5,1,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",한스경제,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4..."
1,2019-05-01,2019,5,1,충청권이 타 지역 지방은행 각축장이 된 까닭,충청투데이,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...
2,2019-05-01,2019,5,1,대전에 문 연 대구은행…지역 반응은 '글쎄',충청투데이,DGB대구은행이 대전지역 진출을 본격화한 가운데 다소 냉담한 반응이 흘러나오고 있다...


In [34]:
df_news.loc[:, ['date']].head(3)

,date
0,2019-05-01
1,2019-05-01
2,2019-05-01


## 정규식 표현 처리

<br>

'[-=+,#/\?:^$.@*\"※%~&ㆍ\‘\’\_, ②外∙·!』『\(\)\[\]\[\]\<\>\'\`<\>\銀…\"\'…》]' 등을 기사제목에서 제거
- tqdm --> 진행률 표시 모듈

**※ 참고 : https://clolee.tistory.com/17, https://codingspooning.tistory.com/138**
<br>

### 정규식 표현 ( 기사 제목 )

In [35]:
from tqdm import tqdm
import copy

# 경고 off
pd.set_option('mode.chained_assignment', None)

# 원본 데이터프레임 복사하여 사용
df_news = copy.deepcopy(df_news)
df_news.insert(5, 'nouns_title', '0')
title_data = df_news['title'].tolist()

# re.sub('[-=+,#/\?:^$.@*\"※%~&ㆍ\‘\’\_, ②外∙·!』『\(\)\[\]\[\]\<\>\'\`<\>\銀…\"\'…》→p”“◇證‧]',
for i in range(len(title_data)):
    cleaner = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·◇②【】<>�→α&]', '', title_data[i])
    df_news['nouns_title'][i] = cleaner

df_news.loc[:5,['title', 'nouns_title']]

,title,nouns_title
0,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",민원 없는 그 날까지 은행권 민원 줄이기에 총력
1,충청권이 타 지역 지방은행 각축장이 된 까닭,충청권이 타 지역 지방은행 각축장이 된 까닭
2,대전에 문 연 대구은행…지역 반응은 '글쎄',대전에 문 연 대구은행지역 반응은 글쎄
3,DGB대구은행 대전 1호 점포 개점...전국 확대,DGB대구은행 대전 1호 점포 개점전국 확대
4,"DGB대구은행, 대전지점 개점",DGB대구은행 대전지점 개점
5,"대구은행, 대전 서구에 '대전지점' 개점",대구은행 대전 서구에 대전지점 개점


### 정규식 표현 ( 기사 내용 )

In [36]:
from tqdm import tqdm
import copy

# 경고 off
pd.set_option('mode.chained_assignment', None)

# 원본 데이터프레임 복사하여 사용
df_news = copy.deepcopy(df_news)
df_news.insert(8, 'nouns_contents', '0')
contents_data = df_news['contents'].tolist()

# re.sub('[-=+,#/\?:^$.@*\"※%~&ㆍ\‘\’\_, ②外∙·!』『\(\)\[\]\[\]\<\>\'\`<\>\銀…\"\'…》→p”“◇證‧]',
for i in range(len(contents_data)):
    cleaner = re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·◇②【】<>�→α&]', '', contents_data[i])
    df_news['nouns_contents'][i] = cleaner

df_news.loc[:5,['contents', 'nouns_contents']]

,contents,nouns_contents
0,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4...",DGB대구은행 59건 SC제일은행 54건 KB국민은행 48건 우리은행 44건으로 뒤...
1,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...
2,DGB대구은행이 대전지역 진출을 본격화한 가운데 다소 냉담한 반응이 흘러나오고 있다...,DGB대구은행이 대전지역 진출을 본격화한 가운데 다소 냉담한 반응이 흘러나오고 있다...
3,DGB대구은행이 대전에 점포를 열었습니다. DGB대구은행은 어제 대전시 서구 문예로...,DGB대구은행이 대전에 점포를 열었습니다 DGB대구은행은 어제 대전시 서구 문예로에...
4,"DGB대구은행이 30일 대전지점(대전광역시 서구 문예로 65)을 개점하고, 국토 및...",DGB대구은행이 30일 대전지점대전광역시 서구 문예로 65을 개점하고 국토 및 교통...
5,대구은행은 30일 대전시 서구에 대전지점을 개점했다. 대구은행은 서울지역 3 개점(...,대구은행은 30일 대전시 서구에 대전지점을 개점했다 대구은행은 서울지역 3 개점서울...


## 토큰화 및 불용어 제거( 1차 )

<br>

**Hannanum / Kkma / Komoran / Mecab / Twitter(Okt, 사용?)**
- 제목을 토큰화 시킴
- Okt형태소 분석기 사용
- norm = True --> 문장을 정규화 Ex) 늘린, 늘린 / 문장 그대로
- stem = True --> 문장을 정규화 Ex) 늘린, 늘리다 / 줄인, 줄이다 / 예뻐서 예뻤다, 예쁘다 / 즉, stem을 사용해야 함. 
- (stem은 stemming 어간화라는 뜻)
<br>

**불용어 : 자연어 처리에 큰 영향을 주지 않는 중요하지 않은 단어들을 말합니다. 주로 조사나 접사 등이 불용어**

In [37]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

In [38]:
# 테스트
print(okt.morphs('늘린 결과이다 줄인 많아서 예뻤다 1위', norm=True))
print(okt.morphs('늘린 결과이다 줄인 많아서 예뻐서 1위', stem=True))

['늘린', '결과', '이다', '줄인', '많아서', '예뻤다', '1', '위']
['늘리다', '결과', '이다', '줄이다', '많다', '예쁘다', '1', '위']


In [39]:
stopwords = ['게', '다', '가', '과', '인', '듯', '지', '으로', '게임', '는', '되',
             '겜', '를', '걍', '한', '에', '은', '좀', '임', '음', '자', '의', '을'
             '잘', '도', '하', '만', '하다', '고', '네', '면', '와', '이', '들']


In [40]:
import konlpy
from konlpy.tag import Okt
okt = Okt()
for sentence in df_news['nouns_title']:
    tmp_X = okt.morphs(sentence, stem=True) # 토큰화
    tmp_X = [word for word in tmp_X if not word in stopwords] # 불용어 제거  
    # print('예외처리 전 :', tmp_X)

    # ['1', '위'] --> ['1위'] 붙이기
    if ('1' in tmp_X) and ('위' in tmp_X):
        for i in range(len(tmp_X)):
            if tmp_X[i] == '위' and tmp_X[i-1] == '1':
                tmp_X[i-1] += tmp_X.pop(i)
                break
    copy_tmp = tmp_X.copy()
    df_news.replace({'nouns_title':sentence}, ' '.join(copy_tmp), inplace=True) # 공백 기준으로 단어 나누기.

In [41]:
df_news.loc[:3, ['title', 'nouns_title']]

,title,nouns_title
0,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",민원 없다 그 날 까지 은행권 민원 줄이다 총력
1,충청권이 타 지역 지방은행 각축장이 된 까닭,충청권 타 지역 지방 은행 각축 장이 되다 까닭
2,대전에 문 연 대구은행…지역 반응은 '글쎄',대전 문 연 대구 은행 지역 반응 글쎄
3,DGB대구은행 대전 1호 점포 개점...전국 확대,DGB 대구 은행 대전 1 호 점포 개점 전국 확대


In [42]:
import konlpy
from konlpy.tag import Okt
okt = Okt()
for sentence in df_news['nouns_contents']:
    tmp_X = okt.morphs(sentence, stem=True) # 토큰화
    tmp_X = [word for word in tmp_X if not word in stopwords] # 불용어 제거
    
    # ['1', '위'] --> ['1위'] 붙이기
    if ('1' in tmp_X) and ('위' in tmp_X):
        for i in range(len(tmp_X)):
            if tmp_X[i] == '위' and tmp_X[i-1] == '1':
                tmp_X[i-1] += tmp_X.pop(i)
                break
    
    copy_tmp = tmp_X.copy()
    df_news.replace({'nouns_contents':sentence}, ' '.join(copy_tmp), inplace=True) # 공백 기준으로 단어 나누기.

In [43]:
df_news.loc[:3, ['contents', 'nouns_contents']]

,contents,nouns_contents
0,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4...",DGB 대구 은행 59 건 SC 제 일 은행 54 건 KB 국민은행 48 건 우리은...
1,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...,DGB 대 구 은행 엊그제 둔산동 대전 지역 1 호 점포 개설 하고 충청권 영업 본...
2,DGB대구은행이 대전지역 진출을 본격화한 가운데 다소 냉담한 반응이 흘러나오고 있다...,DGB 대 구 은행 대전 지역 진출 을 본격 화 가운데 다소 냉담하다 반응 흘러나오...
3,DGB대구은행이 대전에 점포를 열었습니다. DGB대구은행은 어제 대전시 서구 문예로...,DGB 대 구 은행 대전 점포 열다 DGB 대 구 은행 어제 대 전시 서구 문예로 ...


## 파생변수 불필요데이터(이상치?) 처리 
**nouns_contents 컬럼**
- 외화 / 외화 Buyamp / It 해당 단어들이 들어가 있으면 --> 스크래핑 오류 또한, 요약본 기준 글자갯수가 40이하라면 이상치로 판단 --> 또한 의미없는 데이터가 주로 분포됨 90% 이상

In [44]:
r_idx = list()
for i in range(len(df_news)):
    if len(df_news['nouns_contents'][i]) <= 60:# or  '외화 Buyamp' in df_news['nouns_contents'][i]:
        r_idx.append(i)
df_news.loc[r_idx, ['contents', 'nouns_contents']][:3]

,contents,nouns_contents
64,당분간 인수합병(M&amp,당분간 인수합병 Mamp
68,(18:30 대전) KIA(윌랜드) : NC(김영규) (18:30 창원) &lt,1830 대전 KIA 윌 랜드 NC 김영규 1830 창원 lt
1130,김석현 ◆DGB대구은행 &lt,김 석현 ◆ DGB 대구 은행 lt


In [45]:
# 불필요데이터 제거
print('원본 :', len(df_news))
a = len(df_news)
df_news.drop(index = r_idx, axis=0, inplace=True)

원본 : 19405


In [46]:
# 인덱스 번호 새로 부여.
df_news.reset_index(inplace=True)
df_news.drop(['index'], axis=1, inplace=True)

In [47]:
print('제거 후 :', len(df_news))
print('불필요 데이터 :', a - len(df_news))

제거 후 : 19303
불필요 데이터 : 102


## 리스트 변환 --> 단어별로 긍정 부정 중립 판단

In [48]:
df_news.loc[:3, ['nouns_title']]

,nouns_title
0,민원 없다 그 날 까지 은행권 민원 줄이다 총력
1,충청권 타 지역 지방 은행 각축 장이 되다 까닭
2,대전 문 연 대구 은행 지역 반응 글쎄
3,DGB 대구 은행 대전 1 호 점포 개점 전국 확대


In [49]:
df_news.loc[:5, ['nouns_title']]
# nouns 컬럼 --> 공백을 기준으로 나눠 리스트로 변환
df_news['nouns_title'][0].split(' ')
for title_words in df_news['nouns_title'][:4]:
    print(list(title_words.split(' ')))

['민원', '없다', '그', '날', '까지', '은행권', '민원', '줄이다', '총력']
['충청권', '타', '지역', '지방', '은행', '각축', '장이', '되다', '까닭']
['대전', '문', '연', '대구', '은행', '지역', '반응', '글쎄']
['DGB', '대구', '은행', '대전', '1', '호', '점포', '개점', '전국', '확대']


In [50]:
df_news.loc[:3, ['nouns_contents']]

,nouns_contents
0,DGB 대구 은행 59 건 SC 제 일 은행 54 건 KB 국민은행 48 건 우리은...
1,DGB 대 구 은행 엊그제 둔산동 대전 지역 1 호 점포 개설 하고 충청권 영업 본...
2,DGB 대 구 은행 대전 지역 진출 을 본격 화 가운데 다소 냉담하다 반응 흘러나오...
3,DGB 대 구 은행 대전 점포 열다 DGB 대 구 은행 어제 대 전시 서구 문예로 ...


In [51]:
for title_words in df_news['nouns_contents'][:4]:
    print(list(title_words.split(' ')))

['DGB', '대구', '은행', '59', '건', 'SC', '제', '일', '은행', '54', '건', 'KB', '국민은행', '48', '건', '우리은행', '44', '건', '뒤', '이다', '총', '민원', '건수', '국민은행', '1512', '건', '가장', '많다', '이다', 'NH', '농협', '은행', '1250', '건', '신한은행', '1026', '건', '우리은행', '1024', '건', '순', '이다', '고객', '민원', '을']
['DGB', '대', '구', '은행', '엊그제', '둔산동', '대전', '지역', '1', '호', '점포', '개설', '하고', '충청권', '영업', '본격', '나서다', '대', '전', '지점', '이미', '지역', '진출', 'DGB', '금융', '그룹', '계열', '사인', '하이', '투자', '증권', 'DGB', '생명', 'DGB', '캐피탈', '과도', '근접하다', '있다', '전북', '은행', '부산', '은행']
['DGB', '대', '구', '은행', '대전', '지역', '진출', '을', '본격', '화', '가운데', '다소', '냉담하다', '반응', '흘러나오다', '있다', '향토', '은행', '부재', '속', '잇다', '외지', '은행', '진출', '로', '경쟁', '과열', '양상', '을', '띠', '있다', '지역', '기반', '없다', '탓', '외지', '은행', '낮다', '지역']
['DGB', '대', '구', '은행', '대전', '점포', '열다', 'DGB', '대', '구', '은행', '어제', '대', '전시', '서구', '문예로', '대', '전', '지점', '을', '열다', '대', '전', '사', '회복', '공', '동', '모', '금회', '사랑', '나눔', '후', '원금', '천만', '원', '을', '전달', '대', '구', '은행', '대구', '경

In [52]:
# # 동사 중에서 가장 많이 나타나는 20개의 동사 추출 후 기사 검색 
# # 동사는 아직 패스
# from collections import defaultdict

# words = defaultdict(int)

# for title_words in df_news['nouns_contents']:
#     for i in okt.pos(title_words):
# #         if not words:
# #             words[i[1]] = 0
# #         else:
#         len_words[i[1]] += 1
#         if i[1] == 'Verb':
#             words[i[0]] += 1
# #             # if i[0] == '밝히다' or i[0] == '알려지다' or i[0] == '나누다' or i[0] == '늘어나다' or i[0] == '따르다'
# #             print(i[0])
# #     print()


In [53]:
# import operator

# top_20 = sorted(words.items(), key = operator.itemgetter(1), reverse = True)
# check = list()
# for verb_cnt in top_20[2:]:
#     if len(verb_cnt[0]) == 1:
#         continue
#     else:
#         if len(check) <= 20:
#             check.append(verb_cnt[1])
#         else:
#             break

# print(top_20[:25])
# print()
# print(' '.join(map(str, check)))

## 중립 개수가 너무 많음.
- 제목별로 단어개수를 파악해서 긍정단어사전과 부정단어사전을 만들어서 레이블링 작업 진행해야 함. 
- 기사당 긍정단어개수 - 부정단어개수가 0보다 크면 긍정 작으면 부정 0이라면 중립으로 진행

**nouns_title 컬럼에서 길이가 1 / 한자 제거**
<br>
**nouns_title 컬럼에서 'A' / '↑' 예외 처리**

### 기사 제목 ( 추가 불용어 제거 )

In [54]:
# 리스트로 변환 후 체크 
print('리스트 레이블링 정제 전')
# nouns 컬럼 --> 공백을 기준으로 나눠 리스트로 변환
df_news['nouns_title'][0].split(' ')
for title_words in df_news['nouns_title'][:4]:
    print(list(title_words.split(' ')))

리스트 레이블링 정제 전
['민원', '없다', '그', '날', '까지', '은행권', '민원', '줄이다', '총력']
['충청권', '타', '지역', '지방', '은행', '각축', '장이', '되다', '까닭']
['대전', '문', '연', '대구', '은행', '지역', '반응', '글쎄']
['DGB', '대구', '은행', '대전', '1', '호', '점포', '개점', '전국', '확대']


In [55]:
# # [-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·◇②【】<>�→α&]
import re
word="↑" # 테스트 용
# 한글 / 대소문자 / 숫자만 확인
reg = re.compile(r'[가-힣a-zA-Z0-9↑]') 

if reg.match(word):
    print("It is an alphabet")
else:
    print("It is not an alphabet")

It is an alphabet


In [56]:
import re
# 1글자짜리 한글 한자 제거
reg = re.compile(r'[가-힣a-zA-Z0-9↑]')
def Chinese_character(x):
    if reg.match(x): # 한글 / 대소문자 / 숫자
        return False
    else: # 한자
        return True
    
for title_words in df_news['nouns_title']: # xxx --> df_news
    ch_list = list(title_words.split(' '))
    change_value = ''
    for i in range(len(ch_list)):
        if ch_list[i] == '↑' or ch_list[i] == 'A':
            change_value += ch_list[i]
            continue
            
        if len(ch_list[i]) < 2 or Chinese_character(ch_list[i]): # 1글자 이하 또는 함수내에 있다면 제거
            continue
        else:
            change_value += ch_list[i]
        
        # 리스트 마지막 '' 제거
        change_value += ' '
    df_news.replace({'nouns_title' : title_words}, change_value[:-1], inplace=True)
    
df_news.head(2)

,date,year,month,day,title,nouns_title,press,contents,nouns_contents
0,2019-05-01,2019,5,1,"""민원 없는 그 날까지"" 은행권, 민원 줄이기에 총력",민원 없다 까지 은행권 민원 줄이다 총력,한스경제,"DGB대구은행 5.9건, SC제일은행 5.4건, KB국민은행 4.8건, 우리은행 4...",DGB 대구 은행 59 건 SC 제 일 은행 54 건 KB 국민은행 48 건 우리은...
1,2019-05-01,2019,5,1,충청권이 타 지역 지방은행 각축장이 된 까닭,충청권 지역 지방 은행 각축 장이 되다 까닭,충청투데이,DGB대구은행이 엊그제 둔산동에 대전지역 1호 점포를 개설하고 충청권 영업에 본격 ...,DGB 대 구 은행 엊그제 둔산동 대전 지역 1 호 점포 개설 하고 충청권 영업 본...


## 파일 저장

In [57]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19303 entries, 0 to 19302
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            19303 non-null  datetime64[ns]
 1   year            19303 non-null  int64         
 2   month           19303 non-null  int64         
 3   day             19303 non-null  int64         
 4   title           19303 non-null  object        
 5   nouns_title     19303 non-null  object        
 6   press           19303 non-null  object        
 7   contents        19303 non-null  object        
 8   nouns_contents  19303 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 1.3+ MB


### 중복 제거 O

In [58]:
df_news_overlap = df_news.copy()
df_news_overlap.drop_duplicates(['contents'], keep = False, inplace=True)
df_news_overlap.drop_duplicates(['title'], keep = False, inplace=True)

In [59]:
# 저장전 날짜별 내림차순으로 정렬
df_news_overlap.sort_values(by = ['date'], inplace=True, ascending=True)

# 인덱스 번호 새로 부여.
df_news_overlap.reset_index(inplace=True)
df_news_overlap.drop(['index'], axis=1, inplace=True)

In [60]:
len(df_news_overlap)

19303

In [61]:
# 중복 제거 O
df_news_overlap.to_csv('news_data_preprocessing_duplicate.csv')

### 중복 제거 X

In [62]:
# 저장전 날짜별 내림차순으로 정렬
df_news.sort_values(by = ['date'], inplace=True, ascending=True)

# 인덱스 번호 새로 부여.
df_news.reset_index(inplace=True)
df_news.drop(['index'], axis=1, inplace=True)

In [63]:
# 기술 통계량 확인
# df_news.describe(include='all')

In [64]:
# 중복 제거 X
df_news.to_csv('news_data_preprocessing.csv')